**Import needed packages/modules**

In [ ]:
# Cell 1
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import AutoMinorLocator
from scipy.integrate import solve_ivp

import pandas as pd

**Define the model function containing the differential equation:**\
$\large\frac{d^2\theta}{{dt}^2}=-q\frac{d\theta}{dt}-\frac{g}{l}\sin\theta$\
Using linked first order differential equations:

1. $\large\frac{d\omega}{dt}=-q\omega-\frac{g}{l}\sin\theta$

2. $\large\frac{d\theta}{dt}=\omega$

In [ ]:
# Cell 2
def model(time, state_vector, phase_constant, damping_constant):
    # Unpack dependent variables from current state vector
    omega, theta = state_vector
    # Express differential equation
    d_omega = -damping_constant * omega - phase_constant * np.sin(theta)
    d_theta = omega
    return d_omega, d_theta

**Set simulation parameters and initial conditions**
1. The pendulum maintains a constant length of $1.0$ meter
2. The acceleration due to gravity $g=9.81\frac{m}{s^2}$
3. The simulation will last **15** seconds
4. The initial anglular displacement $\theta=75^\circ$
5. The pendulum will be released from _rest_ such that $\omega_0=0.0\;\frac{rad}{s}$
6. The three **damping constants** will be set to their <mark>critical values</mark>

In [ ]:
# Cell 3
pendulum_length = 1.0  # meters
phase_constant = 9.81 / pendulum_length
time_final = 15.0 # seconds
theta_initial = np.radians(75)  # 75 degrees
omega_initial = 0.0 # rad/s

# Set the damping constants
underdamped_constant = 1.0
overdamped_constant = pow(phase_constant, 2) / 2.0
critically_damped_constant = pow(phase_constant, 2) / 4.0

**Use scipy's `solve_ivp()` to numerically estimate the ODE using the RKF45 Method**
1. We will limit the solver to a _maximum_ time step of $0.01$ second
2. The actual time values used will be returned by the solver
3. The solver will return the angular velocity $(\omega)$ at each time value
3. The solver will return the angular displacement $(\theta)$ at each time value

In [ ]:
# Cell 4
sol = solve_ivp(
    model,
    (0, time_final),
    [omega_initial, theta_initial],
    max_step=0.01,
    args=(phase_constant, underdamped_constant),
)
time_steps = sol.t
theta_underdamped = sol.y[1]

sol = solve_ivp(
    model,
    (0, time_final),
    [omega_initial, theta_initial],
    max_step=0.01,
    args=(phase_constant, overdamped_constant),
)
theta_overdamped = sol.y[1]

sol = solve_ivp(
    model,
    (0, time_final),
    [omega_initial, theta_initial],
    max_step=0.01,
    args=(phase_constant, critically_damped_constant),
)
theta_critically_damped = sol.y[1]

# Display the first 10 time and displacement values
pd.DataFrame({
    'Time (s)': time_steps[:10],
    'Under': theta_underdamped[:10],
    'Over': theta_overdamped[:10],
    'Crit': theta_critically_damped[:10],
})

**Plot the angular displacment over time for each type of pendulum**

In [ ]:
# Cell 5
plt.figure(figsize=(8, 6))
plt.plot(time_steps, theta_underdamped,
    label="underdamped", c="r", lw=2, zorder=3)
plt.plot(time_steps, theta_overdamped,
    label="overdamped", c="b", lw=2, zorder=3)
plt.plot(time_steps, theta_critically_damped,
    label="critically damped", c="g", lw=2, zorder=3)
plt.title("Damped Pendulums")
plt.xlabel("Time (sec)")
plt.ylabel("Angular Displacement (radians)")
plt.axhline(y=0.0, color="lightgray")
ax = plt.gca()
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.legend(loc="upper right")
plt.show()